# Cheking data from herds

In [19]:
from datetime import timedelta
import pandas as pd
import numpy as np
import seaborn as sns

Presence in basic files used to create updateDF.csv

In [ ]:
# Function to load, filter, and process data
def process_file(file_path, farm_name, date_column):
    try:
        # Load the data
        df = pd.read_csv(file_path, low_memory=False)
        
        # Filter by farm
        df = df[df["FarmName_Pseudo"].isin(farm_name)]
        
        # Ensure date column is a datetime object
        if date_column in df.columns:
            df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
            # Drop rows with missing dates
            df = df.dropna(subset=[date_column])
        
        print(f"{file_path}: {df.shape}")
        if date_column in df.columns:
            min_value = df[date_column].min()
            max_value = df[date_column].max()
            print(f"Minimum {date_column}: {min_value}")
            print(f"Maximum {date_column}: {max_value}")
        
        return df
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

# Define farm and file paths
farm_name = ["5f7f33d6"] # ["169e580a"] 

# Process each file
calving = process_file("../Data/CowData/calving.csv", farm_name, "CalvingDate")
dry_off = process_file("../Data/CowData/dry_off.csv", farm_name, "DryOffDate")
culling = process_file("../Data/CowData/culling.csv", farm_name, "CullingDate")
breed = process_file("../Data/CowData/breed.csv", farm_name, None)  # No date column exists
insemination = process_file("../Data/CowData/insemination.csv", farm_name, "InseminationDate")
preg_check = process_file("../Data/CowData/pregnancy_checks.csv", farm_name, "PregnancyCheckDate")


Raw MY data for herd from DelPro

In [ ]:
dfmy = pd.read_csv("../Data/CowData/Del_CowMilkYield_Common240823.csv", delimiter=';', low_memory=False)
print(f"No. of milking events in raw data: {dfmy.shape}")  # 5,628,715 x 15
print(dfmy.shape)

# Remove pure duplicates
dfmy = dfmy.drop_duplicates(subset=["SE_Number", "LactationNumber", "StartDate", "StartTime", "SessionNumber", "TotalYield"])
print(f"No. of milking events in raw data: {dfmy.shape}")  # 5,624,788

FarmName_Pseudo = ["5f7f33d6"] # ["169e580a"] # 
lactation_yield = dfmy[dfmy["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
print(f"No. of unique milking records in 5f7f33d6 raw data milking file: {lactation_yield.shape}") 

In [22]:
col_keep = ["FarmName_Pseudo", "SE_Number", "AnimalNumber", "StartDate", "StartTime", "LactationNumber", 
            "DaysInMilk", "SessionNumber", "TotalYield"]
lactation_yield = lactation_yield[col_keep]

Milk yield data from DelPro matched to cows in study

In [ ]:
my = pd.read_csv("../Data/CowData/MY.csv", low_memory=False)
farm = ["5f7f33d6"]
my = my[my["FarmName_Pseudo"].isin(farm)]

print(my.shape)

Cow database

In [ ]:
my = pd.read_csv("../Data/CowData/dfKok.csv", low_memory=False)
farm = ["5f7f33d6"]
my = my[my["FarmName_Pseudo"].isin(farm)]
print(my.shape)

Filtered dataframes
- i.e. data matched within each lactation, within study period 22-01-01 to 2024-08-18 and according to specifications in "HS_MY_FilteringDataframe.ipynb"

In [ ]:
# Filtered dataframe
updateDF = pd.read_csv("../Data/MergedData/MY_weather_filtered.csv", low_memory=False)

farm = ["5f7f33d6"]
updateDF = updateDF[updateDF["FarmName_Pseudo"].isin(farm)]

print(f"No. milk yield records in filtered MY and THI file: {updateDF.shape}")
test = updateDF.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in filtered MY and THI file: {test.shape}")
test = updateDF.drop_duplicates(subset=["SE_Number"])
print(f"No. lactations in filtered MY and THI file: {test.shape}")

In [ ]:
# Finished dataframe with base data from all cows
updateDF = pd.read_csv("../Data/CowData/updateDF.csv", low_memory=False)
farm = ["5f7f33d6"]
updateDF = updateDF[updateDF["FarmName_Pseudo"].isin(farm)]

print(f"No. records in updateDF: {updateDF.shape}")
test = updateDF.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in updateDF: {test.shape}")
test = updateDF.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in updateDF: {test.shape}")
test = updateDF.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in updateDF: {test.shape}")

Raw weather data for study period

In [ ]:
dfw = pd.read_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241009/Gigacow-tools/Projects/HeatStressEvaluation/Data/WeatherData/weather3.csv")

FarmName_Pseudo = ["5f7f33d6"]
dfw = dfw[dfw["FarmName_Pseudo"].isin(FarmName_Pseudo)]
col_keep = ["FarmName_Pseudo", "Tid", "StartDate", "Temperatur", "THI_adj"]
dfw = dfw [col_keep]
dfw

In [ ]:
dfw = dfw.groupby(["FarmName_Pseudo", 'StartDate']).agg({
    'Temperatur': 'mean',
    "THI_adj": "mean"
}).reset_index()

dfw

In [ ]:
# Load data
df = pd.read_csv("your_data.csv", low_memory=False)

# Checking missing data
# Iterate through all columns
for column in df.columns:
    # Count non-NaN values
    non_nan_count = df[column].notna().sum()
    # Count NaN values
    is_nan_count = df[column].isna().sum()
    # Print results
    print(f"Column: {column}")
    print(f"No. of non-NaN values: {non_nan_count}")
    print(f"No. of NaN values: {is_nan_count}")
    print()